In [17]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

CHECKPOINT_PATH = "./results/checkpoint-12500"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained(CHECKPOINT_PATH, num_labels=2)
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model on {device}")

Model on cuda


In [29]:
from datasets import load_dataset
from torch.utils.data import DataLoader

val = load_dataset("glue", "sst2", split="validation")

def tokenize_func(x):
    return tokenizer(x['sentence'], padding="max_length", max_length=128, truncation=True)

tokenized_dataset = val.map(tokenize_func, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

dataloader = DataLoader(tokenized_dataset, batch_size=4)

Reusing dataset glue (/home/jupyter/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [1]:
import torch
torch.cuda.empty_cache()

In [31]:
from sklearn.metrics import accuracy_score

model.eval()
preds, labels = [], []
for batch in dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    label = batch['label'].to(device)
    
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    pred = torch.argmax(logits, dim=-1)
    preds.extend(pred.cpu().numpy().tolist())
    labels.extend(label.cpu().numpy().tolist())
    
acc = accuracy_score(labels, preds)
print(f"SST-2 Validation Accuracy: {acc * 100:.2f}%")

SST-2 Validation Accuracy: 91.51%


In [52]:
def predict_sentence(sentence):
    tokenized = tokenizer(
        [sentence],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    input_ids, attention_mask = tokenized['input_ids'], tokenized['attention_mask']
    input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
    
    prediction = torch.argmax(logits, dim=-1).cpu().numpy().tolist()[0]
    confidence = torch.softmax(logits, dim=-1).cpu().numpy().tolist()[0]
    conf_value = confidence[prediction]
    
    if prediction == 1:
        print(f"Sentiment of sentence was positive with confidence {conf_value * 100}")
    else:
        print(f"Sentiment of sentence was negative with confidence {conf_value * 100}")
        
    return prediction, conf_value

In [53]:
pred, confidence = predict_sentence('I absolutely loved that movie!')

Sentiment of sentence was positive with confidence 98.73080849647522


In [54]:
pred, confidence = predict_sentence('That movie was terrible, what was that, not sure what the director was thinking')

Sentiment of sentence was negative with confidence 99.87925291061401


In [55]:
predict_sentence('Hmm, what do you want to eat today?')

Sentiment of sentence was negative with confidence 96.30213975906372


(0, 0.9630213975906372)

In [56]:
predict_sentence('What\'s the weather like tomorrow?')

Sentiment of sentence was negative with confidence 90.63577055931091


(0, 0.9063577055931091)

In [57]:
predict_sentence('Eh, yeah that movie was okay. It wasn\'t bad, but definitely not going to watch that again')

Sentiment of sentence was negative with confidence 79.75066304206848


(0, 0.7975066304206848)